## Universidad del Valle de Guatemala
## CC3094 - Security Data Science - Sección 10
## Proyecto Final - DDoS Detection
## Integrantes:
## - 19402 - Julio Herrera
## - 19270 - Oliver De León
## - 18341 - Randy Venegas

## Fase 1: Propuesta de Proyecto

La propuesta de proyecto aprobada se encuentra en el documento Avances Proyecto Security Data Science.pdf donde se define la motivación por la que se seleccionó este proyecto, las preguntas clave a considerar durante el desarrollo, la revisión de literatura actual sobre el tema  y la recolección de datos.

Como resumen, el proyecto seleccionado fue desarrollar un sistema de detección de ataques DDoS, específicamente de llamadas a un servidor que sean parte de un ataque DDoS, esto con el propósito de que organizaciones y empresas puedan detectar y mitigar estos ataques enfocados a sus servidores, obteniendo una precisión y recall igual o mayor al que se ha demostrado en la literatura actual.

## Fase 2: Implementación

Esta fase la cual se desarrolla en este jupyter notebook, consiste en la implementación completa del modelo, pasando por la limpieza de datos y selección de features relevantes, pre-procesamiento y análisis exploratorio, implementación de los modelos y evaluación.

### Pre-procesamiento

Para el pre-procesamiento se cargarán los distintos datasets necesarios, se juntarán, se limpiarán y se seleccionarán las features relevantes para el modelo, el objetivo de ese pre-procesamiento no es dejar el dataset final sino tener un dataset manejable, sin features que de primeras sabemos que no son relevantes para ir haciendo la exploración luego.

#### Carga de los datasets

El dataset completo es DDoS Evaluation Dataset (CIC-DDoS2019) de la University of New Brunswick (UNB) y se encuentra disponible en https://www.unb.ca/cic/datasets/ddos-2019.html o la copia personal en https://drive.google.com/file/d/1U9ccgLqrv36eLZuV0-pgUsGshgqJ73ma/view?usp=sharing que contiene el zip CSV-01-12 (HASH b86b3553b1c5086222b27ea17a27e07a) y la copia en https://drive.google.com/file/d/1v88DSdQ-tmwICOQWZGXUpKIj9JJD3ji6/view?usp=sharing que contiene el zip CSV-03-11 (HASH f43510dfa38483cb5851063807997baa). Estos dataset contienen multiples archivos csv que corresponden los primeros (CSV-01-12) a 12 tipos de ataques DDoS incluyendo NTP, DNS, LDAP, MSSQL, NetBIOS, SNMP, SSDP, UDP, UDP-Lag, WebDDoS, SYN y TFTP durante el día de entrenamiento y los otros (CSV-03-11) a 7 tipos de ataques incluyendo PortScan, NetBIOS, LDAP, MSSQL, UDP, UDP-Lag y SYN durante el día de testeo.

In [936]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn import svm
from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm_notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [937]:
# train day
'''DrDoS_DNS = pd.read_csv('datasets/DrDoS_DNS.csv')
DrDoS_LDAP = pd.read_csv('datasets/DrDoS_LDAP.csv')
DrDoS_MSSQL = pd.read_csv('datasets/DrDoS_MSSQL.csv')
DrDoS_NetBIOS = pd.read_csv('datasets/DrDoS_NetBIOS.csv')
DrDoS_NTP = pd.read_csv('datasets/DrDoS_NTP.csv')
DrDoS_SNMP = pd.read_csv('datasets/DrDoS_SNMP.csv')
DrDoS_SSDP = pd.read_csv('datasets/DrDoS_SSDP.csv')
DrDoS_UDP = pd.read_csv('datasets/DrDoS_UDP.csv')
Syn = pd.read_csv('datasets/Syn1.csv')
TFTPdf = pd.read_csv('datasets/TFTP.csv')
UDPLag = pd.read_csv('datasets/UDPLag1.csv')

# test day
LDAPdf = pd.read_csv('datasets/LDAP.csv')
MSSQLdf = pd.read_csv('datasets/MSSQL.csv')
NetBIOS = pd.read_csv('datasets/NetBIOS.csv')
Portmap = pd.read_csv('datasets/Portmap.csv')
Syn_2 = pd.read_csv('datasets/Syn.csv')
UDPdf = pd.read_csv('datasets/UDP.csv')
UDPLag_2 = pd.read_csv('datasets/UDPLag.csv')'''
DrDoS_DNS = pd.read_csv('data/CSV-01-12/01-12/DrDoS_DNS.csv', nrows=100000)
DrDoS_LDAP = pd.read_csv('data/CSV-01-12/01-12/DrDoS_LDAP.csv', nrows=100000)
DrDoS_MSSQL = pd.read_csv('data/CSV-01-12/01-12/DrDoS_MSSQL.csv', nrows=100000)
DrDoS_NetBIOS = pd.read_csv('data/CSV-01-12/01-12/DrDoS_NetBIOS.csv', nrows=100000)
DrDoS_NTP = pd.read_csv('data/CSV-01-12/01-12/DrDoS_NTP.csv', nrows=100000)
DrDoS_SNMP = pd.read_csv('data/CSV-01-12/01-12/DrDoS_SNMP.csv', nrows=100000)
DrDoS_SSDP = pd.read_csv('data/CSV-01-12/01-12/DrDoS_SSDP.csv', nrows=100000)
DrDoS_UDP = pd.read_csv('data/CSV-01-12/01-12/DrDoS_UDP.csv', nrows=100000)
Syn = pd.read_csv('data/CSV-01-12/01-12/Syn.csv', nrows=100000)
TFTPdf = pd.read_csv('data/CSV-01-12/01-12/TFTP.csv', nrows=100000)
UDPLag = pd.read_csv('data/CSV-01-12/01-12/UDPLag.csv', nrows=100000)

# test day
LDAPdf = pd.read_csv('data/CSV-03-11/03-11/LDAP.csv', nrows=100000)
MSSQLdf = pd.read_csv('data/CSV-03-11/03-11/MSSQL.csv', nrows=100000)
NetBIOS = pd.read_csv('data/CSV-03-11/03-11/NetBIOS.csv', nrows=100000)
Portmap = pd.read_csv('data/CSV-03-11/03-11/Portmap.csv', nrows=100000)
Syn_2 = pd.read_csv('data/CSV-03-11/03-11/Syn.csv', nrows=100000)
UDPdf = pd.read_csv('data/CSV-03-11/03-11/UDP.csv', nrows=100000)
UDPLag_2 = pd.read_csv('data/CSV-03-11/03-11/UDPLag.csv', nrows=100000)

C:\Users\julio\AppData\Local\Temp\ipykernel_20036\3735033965.py:22: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  DrDoS_DNS = pd.read_csv('data/CSV-01-12/01-12/DrDoS_DNS.csv', nrows=100000)
C:\Users\julio\AppData\Local\Temp\ipykernel_20036\3735033965.py:23: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  DrDoS_LDAP = pd.read_csv('data/CSV-01-12/01-12/DrDoS_LDAP.csv', nrows=100000)
C:\Users\julio\AppData\Local\Temp\ipykernel_20036\3735033965.py:24: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  DrDoS_MSSQL = pd.read_csv('data/CSV-01-12/01-12/DrDoS_MSSQL.csv', nrows=100000)
C:\Users\julio\AppData\Local\Temp\ipykernel_20036\3735033965.py:25: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  DrDoS_NetBIOS = pd.read_csv('data/CSV-01-12/01-12/DrDoS_NetBIOS.csv', nrows=100000

#### Vistazo a los datasets

Se da un vistazo rápido a los datasets para ver la cantidad de datos que se tienen y la cantidad de features que se tienen.

In [938]:
## check the data

DrDoS_DNS.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,425,172.16.0.5-192.168.50.1-634-60495-17,172.16.0.5,634,192.168.50.1,60495,17,2018-12-01 10:51:39.813448,28415,97,0,42680.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,1.502024e+06,3413.689952,295.989583,500.959301,3596.0,1.0,28415.0,295.989583,500.959301,3596.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-97,0,3413.689952,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,444.536082,440.0,0.0,-97,0,0,0,0,0,0,97,42680,0,0,-1,-1,96,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
1,430,172.16.0.5-192.168.50.1-60495-634-17,192.168.50.1,634,172.16.0.5,60495,17,2018-12-01 10:51:39.820842,2,2,0,880.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,4.400000e+08,1000000.000000,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.000000,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,660.000000,440.0,0.0,-2,0,0,0,0,0,0,2,880,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DrDoS_DNS
2,1654,172.16.0.5-192.168.50.1-634-46391-17,172.16.0.5,634,192.168.50.1,46391,17,2018-12-01 10:51:39.852499,48549,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,1.812602e+06,4119.549321,243.964824,578.101371,5418.0,1.0,48549.0,243.964824,578.101371,5418.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-200,0,4119.549321,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,-200,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
3,2927,172.16.0.5-192.168.50.1-634-11894-17,172.16.0.5,634,192.168.50.1,11894,17,2018-12-01 10:51:39.890213,48337,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,1.820552e+06,4137.617146,242.899497,485.292695,3337.0,1.0,48337.0,242.899497,485.292695,3337.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-200,0,4137.617146,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,-200,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
4,694,172.16.0.5-192.168.50.1-634-27878-17,172.16.0.5,634,192.168.50.1,27878,17,2018-12-01 10:51:39.941151,32026,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,2.747767e+06,6244.925998,160.934673,196.891271,1236.0,0.0,32026.0,160.934673,196.891271,1236.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-200,0,6244.925998,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,-200,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS


In [939]:
DrDoS_LDAP.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,21010,172.16.0.5-192.168.50.1-0-0-0,172.16.0.5,0,192.168.50.1,0,0,2018-12-01 11:22:40.254769,9141643,85894,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,9.398967e+03,106.39591,209.905159,2968.0,0.0,9141643.0,106.430594,209.94653,2968.0,0.0,8487477.0,314351.0,1.147263e+06,5975703.0,0.0,0,0,0,0,0,0,9.395904e+03,3.062907,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,85894,0,28,0,-1,-1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP
1,20932,172.16.0.5-192.168.50.1-900-1808-17,172.16.0.5,900,192.168.50.1,1808,17,2018-12-01 11:22:40.255361,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2.944000e+09,2.000000e+06,1.00000,0.000000,1.0,1.0,1.0,1.000000,0.00000,1.0,1.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP
2,27876,172.16.0.5-192.168.50.1-900-58766-17,172.16.0.5,900,192.168.50.1,58766,17,2018-12-01 11:22:40.255568,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1.000000e+06,2.00000,0.000000,2.0,2.0,2.0,2.000000,0.00000,2.0,2.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP
3,24270,172.16.0.5-192.168.50.1-900-35228-17,172.16.0.5,900,192.168.50.1,35228,17,2018-12-01 11:22:40.256113,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2.944000e+09,2.000000e+06,1.00000,0.000000,1.0,1.0,1.0,1.000000,0.00000,1.0,1.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP
4,5109,172.16.0.5-192.168.50.1-900-44969-17,172.16.0.5,900,192.168.50.1,44969,17,2018-12-01 11:22:40.256285,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1.000000e+06,2.00000,0.000000,2.0,2.0,2.0,2.000000,0.00000,2.0,2.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP


In [940]:
DrDoS_MSSQL.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,139,172.16.0.5-192.168.50.1-0-0-0,172.16.0.5,0,192.168.50.1,0,0,2018-12-01 11:32:32.915441,119151083,60959,462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,5.154884e+02,1939.939482,65221.682111,5975622.0,0.0,119151083.0,1954.642262,65471.192742,5975749.0,0.0,114024380.0,247341.388286,747004.839439,5975722.0,0.0,0,0,0,0,0,0,5.116110e+02,3.87743,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,60959,0,462,0,-1,-1,0,0,16786536.6,2.853681e+07,67834732.0,4024278.0,5975510.0,98.183502,5975622.0,5975358.0,0,1,DrDoS_MSSQL
1,38385,172.16.0.5-192.168.50.1-850-20345-17,172.16.0.5,850,192.168.50.1,20345,17,2018-12-01 11:32:32.915442,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2.944000e+09,2.000000e+06,1.000000,0.000000,1.0,1.0,1.0,1.000000,0.000000,1.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.00000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_MSSQL
2,27033,172.16.0.5-192.168.50.1-851-21631-17,172.16.0.5,851,192.168.50.1,21631,17,2018-12-01 11:32:32.915578,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2.944000e+09,2.000000e+06,1.000000,0.000000,1.0,1.0,1.0,1.000000,0.000000,1.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.00000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_MSSQL
3,34348,172.16.0.5-192.168.50.1-852-15332-17,172.16.0.5,852,192.168.50.1,15332,17,2018-12-01 11:32:32.915773,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1.000000e+06,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.00000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_MSSQL
4,19225,172.16.0.5-192.168.50.1-853-41853-17,172.16.0.5,853,192.168.50.1,41853,17,2018-12-01 11:32:32.916114,2,2,0,2848.0,0.0,1424.0,1424.0,1424.0,0.0,0.0,0.0,0.0,0.0,1.424000e+09,1.000000e+06,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.00000,1424.0,1424.0,1424.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2136.0,1424.0,0.0,-2,0,0,0,0,0,0,2,2848,0,0,-1,-1,1,-1,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_MSSQL


In [941]:
DrDoS_NetBIOS.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,256565,172.16.0.5-192.168.50.1-34012-2334-17,172.16.0.5,34012,192.168.50.1,2334,17,2018-12-01 11:47:08.463789,1,2,0,884.0,0.0,442.0,442.0,442.0,0.0,0.0,0.0,0.0,0.0,8.840000e+08,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,442.0,442.0,442.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,663.0,442.0,0.0,-2,0,0,0,0,0,0,2,884,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NetBIOS
1,252918,172.16.0.5-192.168.50.1-34013-50170-17,172.16.0.5,34013,192.168.50.1,50170,17,2018-12-01 11:47:08.464316,1,2,0,2346.0,0.0,1173.0,1173.0,1173.0,0.0,0.0,0.0,0.0,0.0,2.346000e+09,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,1173.0,1173.0,1173.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,1759.5,1173.0,0.0,-2,0,0,0,0,0,0,2,2346,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NetBIOS
2,174257,172.16.0.5-192.168.50.1-34014-61534-17,172.16.0.5,34014,192.168.50.1,61534,17,2018-12-01 11:47:08.464472,1,2,0,942.0,0.0,471.0,471.0,471.0,0.0,0.0,0.0,0.0,0.0,9.420000e+08,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,471.0,471.0,471.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,706.5,471.0,0.0,-2,0,0,0,0,0,0,2,942,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NetBIOS
3,185193,172.16.0.5-192.168.50.1-34015-8930-17,172.16.0.5,34015,192.168.50.1,8930,17,2018-12-01 11:47:08.464520,1,2,0,1020.0,0.0,510.0,510.0,510.0,0.0,0.0,0.0,0.0,0.0,1.020000e+09,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,510.0,510.0,510.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,765.0,510.0,0.0,-2,0,0,0,0,0,0,2,1020,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NetBIOS
4,198671,172.16.0.5-192.168.50.1-34016-33040-17,172.16.0.5,34016,192.168.50.1,33040,17,2018-12-01 11:47:08.464925,2,2,0,1002.0,0.0,501.0,501.0,501.0,0.0,0.0,0.0,0.0,0.0,5.010000e+08,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,501.0,501.0,501.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,751.5,501.0,0.0,-2,0,0,0,0,0,0,2,1002,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NetBIOS


In [942]:
DrDoS_NTP.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,0,172.16.0.5-192.168.50.1-60675-80-6,172.16.0.5,60675,192.168.50.1,80,6,2018-12-01 09:17:11.183810,5220876,12,9,1928.0,4290.0,509.0,0.0,160.666667,237.878247,1073.0,0.0,476.666667,565.257242,1190.987873,4.022313,2.610438e+05,1.117421e+06,5004919.0,2.0,215957.0,1.963245e+04,5.779980e+04,192825.0,2.0,5220754.0,652594.25,1.759944e+06,5005096.0,3.0,0,0,0,0,400,304,2.298465,1.723849,0.0,1073.0,282.636364,424.028587,179800.242424,0,0,0,0,1,0,0,0,0.0,296.095238,160.666667,476.666667,400,0,0,0,0,0,0,12,1928,9,4290,65535,243,4,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,205.174.165.72/c.php,1,DrDoS_NTP
1,7,172.16.0.5-192.168.50.1-60676-80-6,172.16.0.5,60676,192.168.50.1,80,6,2018-12-01 09:17:11.205636,12644252,5,2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.553611,2.107375e+06,5.161910e+06,12644080.0,1.0,12644252.0,3.161063e+06,6.322011e+06,12644080.0,1.0,1.0,1.00,0.000000e+00,1.0,1.0,0,0,0,0,176,80,0.395437,0.158175,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,176,0,0,0,0,0,0,5,0,2,0,65535,28960,0,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP
2,12858,192.168.50.7-65.55.163.78-50458-443-6,65.55.163.78,443,192.168.50.7,50458,6,2018-12-01 09:17:12.634569,3,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,666666.666667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,40,0,666666.666667,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0.0,0.000000,0.000000,0.000000,40,0,0,0,0,0,0,2,0,0,0,0,-1,0,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,BENIGN
3,10191,192.168.50.7-65.55.163.78-50465-443-6,65.55.163.78,443,192.168.50.7,50465,6,2018-12-01 09:17:13.458370,3,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,666666.666667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,40,0,666666.666667,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0.0,0.000000,0.000000,0.000000,40,0,0,0,0,0,0,2,0,0,0,0,-1,0,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,BENIGN
4,239,192.168.50.253-224.0.0.5-0-0-0,192.168.50.253,0,224.0.0.5,0,0,2018-12-01 09:17:13.470913,114329232,52,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.454827,2.241750e+06,4.083247e+06,9950741.0,1.0,114329232.0,2.241750e+06,4.083247e+06,9950741.0,1.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0.454827,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0,0,0,0,

In [943]:
DrDoS_SNMP.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,109857,172.16.0.5-192.168.50.1-528-47330-17,172.16.0.5,528,192.168.50.1,47330,17,2018-12-01 12:00:13.902782,2,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,229000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SNMP
1,238192,172.16.0.5-192.168.50.1-529-26888-17,172.16.0.5,529,192.168.50.1,26888,17,2018-12-01 12:00:13.902785,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,458000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SNMP
2,82963,172.16.0.5-192.168.50.1-530-3723-17,172.16.0.5,530,192.168.50.1,3723,17,2018-12-01 12:00:13.903230,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,458000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SNMP
3,205787,172.16.0.5-192.168.50.1-531-38862-17,172.16.0.5,531,192.168.50.1,38862,17,2018-12-01 12:00:13.903311,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,458000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SNMP
4,83411,172.16.0.5-192.168.50.1-532-35383-17,172.16.0.5,532,192.168.50.1,35383,17,2018-12-01 12:00:13.903490,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,458000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SNMP


In [944]:
DrDoS_SSDP.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,72,172.16.0.5-192.168.50.1-0-0-0,172.16.0.5,0,192.168.50.1,0,0,2018-12-01 12:23:13.663425,119714230,49476,214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,4.150718e+02,2409.270261,36195.71635,2001059.0,0.0,119714230.0,2419.691359,36275.32503,2001059.0,0.0,114024102.0,535324.422535,1.232162e+06,5976064.0,1.0,0,0,0,0,0,0,4.132842e+02,1.78759,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,49476,0,214,0,-1,-1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP
1,55171,172.16.0.5-192.168.50.1-700-36081-17,172.16.0.5,700,192.168.50.1,36081,17,2018-12-01 12:23:13.663475,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1.000000e+06,2.000000,0.00000,2.0,2.0,2.0,2.000000,0.00000,2.0,2.0,0.0,0.000000,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.00000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP
2,39545,172.16.0.5-192.168.50.1-701-25269-17,172.16.0.5,701,192.168.50.1,25269,17,2018-12-01 12:23:13.663526,1,2,0,2604.0,0.0,1302.0,1302.0,1302.0,0.0,0.0,0.0,0.0,0.0,2.604000e+09,2.000000e+06,1.000000,0.00000,1.0,1.0,1.0,1.000000,0.00000,1.0,1.0,0.0,0.000000,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.00000,1302.0,1302.0,1302.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,1953.0,1302.0,0.0,-2,0,0,0,0,0,0,2,2604,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP
3,20334,172.16.0.5-192.168.50.1-702-2533-17,172.16.0.5,702,192.168.50.1,2533,17,2018-12-01 12:23:13.663622,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1.000000e+06,2.000000,0.00000,2.0,2.0,2.0,2.000000,0.00000,2.0,2.0,0.0,0.000000,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.00000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP
4,18397,172.16.0.5-192.168.50.1-703-34942-17,172.16.0.5,703,192.168.50.1,34942,17,2018-12-01 12:23:13.663844,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1.000000e+06,2.000000,0.00000,2.0,2.0,2.0,2.000000,0.00000,2.0,2.0,0.0,0.000000,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,1.000000e+06,0.00000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP


In [945]:
DrDoS_UDP.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,101418,172.16.0.5-192.168.50.1-43443-6652-17,172.16.0.5,43443,192.168.50.1,6652,17,2018-12-01 12:36:57.628026,218395,6,0,2088.0,0.0,393.0,321.0,348.0,35.088460,0.0,0.0,0.0,0.0,9.560658e+03,2.747316e+01,43679.000000,59812.246961,110075.0,0.0,218395.0,43679.000000,59812.246961,110075.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-6,0,2.747316e+01,0.0,321.0,393.0,344.142857,33.617597,1130.142857,0,0,0,0,0,0,0,0,0.0,401.50,348.0,0.0,-6,0,0,0,0,0,0,6,2088,0,0,-1,-1,5,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
1,21564,172.16.0.5-192.168.50.1-54741-9712-17,172.16.0.5,54741,192.168.50.1,9712,17,2018-12-01 12:36:57.628076,108219,4,0,1398.0,0.0,369.0,330.0,349.5,22.516660,0.0,0.0,0.0,0.0,1.291825e+04,3.696209e+01,36073.000000,62478.536731,108217.0,1.0,108219.0,36073.000000,62478.536731,108217.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-4,0,3.696209e+01,0.0,330.0,369.0,345.600000,21.361180,456.300000,0,0,0,0,0,0,0,0,0.0,432.00,349.5,0.0,-4,0,0,0,0,0,0,4,1398,0,0,-1,-1,3,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
2,23389,172.16.0.5-192.168.50.1-56589-4680-17,172.16.0.5,56589,192.168.50.1,4680,17,2018-12-01 12:36:57.628164,104579,4,0,1438.0,0.0,389.0,330.0,359.5,34.063666,0.0,0.0,0.0,0.0,1.375037e+04,3.824860e+01,34859.666667,60376.981751,104577.0,1.0,104579.0,34859.666667,60376.981751,104577.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-4,0,3.824860e+01,0.0,330.0,389.0,353.600000,32.315631,1044.300000,0,0,0,0,0,0,0,0,0.0,442.00,359.5,0.0,-4,0,0,0,0,0,0,4,1438,0,0,-1,-1,3,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
3,48872,172.16.0.5-192.168.50.1-40233-2644-17,172.16.0.5,40233,192.168.50.1,2644,17,2018-12-01 12:36:57.628166,110967,4,0,1544.0,0.0,389.0,383.0,386.0,3.464102,0.0,0.0,0.0,0.0,1.391405e+04,3.604675e+01,36989.000000,64065.095270,110965.0,1.0,110967.0,36989.000000,64065.095270,110965.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-4,0,3.604675e+01,0.0,383.0,389.0,386.600000,3.286335,10.800000,0,0,0,0,0,0,0,0,0.0,483.25,386.0,0.0,-4,0,0,0,0,0,0,4,1544,0,0,-1,-1,3,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
4,80354,172.16.0.5-192.168.50.1-33989-16901-17,172.16.0.5,33989,192.168.50.1,16901,17,2018-12-01 12:36:57.628217,1,2,0,766.0,0.0,383.0,383.0,383.0,0.000000,0.0,0.0,0.0,0.0,7.660000e+08,2.000000e+06,1.000000,0.000000,1.0,1.0,1.0,1.000000,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,383.0,383.0,383.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,574.50,383.0,0.0,-2,0,0,0,0,0,0,2,766,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP


In [946]:
Syn.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,281052,172.16.0.5-192.168.50.1-53058-53058-6,172.16.0.5,53058,192.168.50.1,53058,6,2018-12-01 13:30:30.741451,115799309,19,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.813482e-01,5.789965e+06,7.370247e+06,21714933.0,0.0,115799309.0,6.433295e+06,7.505277e+06,21714933.0,0.0,48.0,48.0,0.0,48.0,48.0,0,0,0,0,380,40,1.640770e-01,0.017271,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,380,0,0,0,0,0,0,19,0,2,0,5840,0,0,20,244280.142857,646237.483665,1709809.0,1.0,1.426117e+07,3.220326e+06,21714933.0,11043464.0,0,1,Syn
1,450424,172.16.0.5-192.168.50.1-32237-32237-6,172.16.0.5,32237,192.168.50.1,32237,6,2018-12-01 13:30:30.741452,113973933,16,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.403830e-01,7.598262e+06,8.575157e+06,20019405.0,0.0,113973933.0,7.598262e+06,8.575157e+06,20019405.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,320,0,1.403830e-01,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,320,0,0,0,0,0,0,16,0,0,0,5840,-1,0,20,9.000000,19.595918,49.0,1.0,1.628198e+07,2.573891e+06,20019405.0,11993631.0,0,1,Syn
2,182979,172.16.0.5-192.168.50.1-60495-9840-6,172.16.0.5,60495,192.168.50.1,9840,6,2018-12-01 13:30:30.741501,112,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.571429e+04,3.733333e+01,6.379916e+01,111.0,0.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,40,1.785714e+04,17857.142857,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0,40,0,0,0,0,0,0,2,0,2,0,5840,0,0,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
3,41540,172.16.0.5-192.168.50.1-59724-59724-6,172.16.0.5,59724,192.168.50.1,59724,6,2018-12-01 13:30:30.741563,105985004,16,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.509648e-01,7.065667e+06,8.074003e+06,20954123.0,1.0,105985004.0,7.065667e+06,8.074003e+06,20954123.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,320,0,1.509648e-01,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,320,0,0,0,0,0,0,16,0,0,0,5840,-1,0,20,7.857143,17.705259,48.0,1.0,1.514071e+07,3.077366e+06,20954123.0,11120336.0,0,1,Syn
4,358711,172.16.0.5-192.168.50.1-60496-32538-6,172.16.0.5,60496,192.168.50.1,32538,6,2018-12-01 13:30:30.741565,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,40,0,0,0,0,0,0,2,0,0,0,5840,-1,0,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn


In [947]:
TFTPdf.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,46970,172.16.0.5-192.168.50.1-64670-64670-6,172.16.0.5,64670,192.168.50.1,64670,6,2018-12-01 13:34:27.403713,13120080,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304876,4.373360e+06,7.574880e+06,13120078.0,1.0,13120080.0,4.373360e+06,7.574880e+06,13120078.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,80,0,0.304876,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,80,0,0,0,0,0,0,4,0,0,0,5840,-1,0,20,1.0,0.0,1.0,1.0,13120078.0,0.0,13120078.0,13120078.0,0,1,TFTP
1,164775,172.16.0.5-192.168.50.1-16405-16405-6,172.16.0.5,16405,192.168.50.1,16405,6,2018-12-01 13:34:27.404124,3990342,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.002420,1.330114e+06,2.303823e+06,3990340.0,1.0,3990342.0,1.330114e+06,2.303823e+06,3990340.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,80,0,1.002420,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,80,0,0,0,0,0,0,4,0,0,0,5840,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
2,177893,172.16.0.5-192.168.50.1-45838-45838-6,172.16.0.5,45838,192.168.50.1,45838,6,2018-12-01 13:34:27.404125,11842236,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337774,3.947412e+06,6.837116e+06,11842234.0,1.0,11842236.0,3.947412e+06,6.837116e+06,11842234.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,80,0,0.337774,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,80,0,0,0,0,0,0,4,0,0,0,5840,-1,0,20,1.0,0.0,1.0,1.0,11842234.0,0.0,11842234.0,11842234.0,0,1,TFTP
3,82756,172.16.0.5-192.168.50.1-27049-27049-6,172.16.0.5,27049,192.168.50.1,27049,6,2018-12-01 13:34:27.404177,3579644,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.117430,1.193215e+06,2.066708e+06,3579643.0,0.0,3579644.0,1.193215e+06,2.066708e+06,3579643.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,80,0,1.117430,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,80,0,0,0,0,0,0,4,0,0,0,5840,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
4,153782,172.16.0.5-192.168.50.1-43305-27049-6,172.16.0.5,43305,192.168.50.1,27049,6,2018-12-01 13:34:27.404228,90,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44444.444444,3.000000e+01,5.022947e+01,88.0,1.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0,0,0,0,40,40,22222.222222,22222.222222,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0,40,0,0,0,0,0,0,2,0,2,0,5840,0,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP


In [948]:
UDPLag.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,186059,172.16.0.5-192.168.50.1-58445-4463-17,172.16.0.5,58445,192.168.50.1,4463,17,2018-12-01 13:04:45.928673,1,2,0,766.0,0.0,383.0,383.0,383.0,0.0,0.0,0.0,0.0,0.0,766000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,383.0,383.0,383.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,574.5,383.0,0.0,-2,0,0,0,0,0,0,2,766,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
1,135692,172.16.0.5-192.168.50.1-36908-9914-17,172.16.0.5,36908,192.168.50.1,9914,17,2018-12-01 13:04:45.928913,1,2,0,778.0,0.0,389.0,389.0,389.0,0.0,0.0,0.0,0.0,0.0,778000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,389.0,389.0,389.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,583.5,389.0,0.0,-2,0,0,0,0,0,0,2,778,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
2,33822,172.16.0.5-192.168.50.1-41727-32361-17,172.16.0.5,41727,192.168.50.1,32361,17,2018-12-01 13:04:45.928915,2,2,0,750.0,0.0,375.0,375.0,375.0,0.0,0.0,0.0,0.0,0.0,375000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,375.0,375.0,375.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,562.5,375.0,0.0,-2,0,0,0,0,0,0,2,750,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
3,24498,172.16.0.5-192.168.50.1-55447-5691-17,172.16.0.5,55447,192.168.50.1,5691,17,2018-12-01 13:04:45.929024,2,2,0,738.0,0.0,369.0,369.0,369.0,0.0,0.0,0.0,0.0,0.0,369000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,369.0,369.0,369.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,553.5,369.0,0.0,-2,0,0,0,0,0,0,2,738,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
4,117372,172.16.0.5-192.168.50.1-58794-56335-17,172.16.0.5,58794,192.168.50.1,56335,17,2018-12-01 13:04:45.929096,1,2,0,750.0,0.0,375.0,375.0,375.0,0.0,0.0,0.0,0.0,0.0,750000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,375.0,375.0,375.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,562.5,375.0,0.0,-2,0,0,0,0,0,0,2,750,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag


In [949]:
LDAPdf.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,13605,172.16.0.5-192.168.50.4-870-2908-17,172.16.0.5,870,192.168.50.4,2908,17,2018-11-03 10:09:00.565557,1,2,0,422.0,0.0,211.0,211.0,211.0,0.0,0.0,0.0,0.0,0.0,4.220000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,211.0,211.0,211.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,316.5,211.0,0.0,-2,0,0,0,0,0,0,2,422,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
1,62631,172.16.0.5-192.168.50.4-871-53796-17,172.16.0.5,871,192.168.50.4,53796,17,2018-11-03 10:09:00.565559,48,2,0,530.0,0.0,265.0,265.0,265.0,0.0,0.0,0.0,0.0,0.0,1.104167e+07,4.166667e+04,48.0,0.0,48.0,48.0,48.0,48.0,0.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,4.166667e+04,0.0,265.0,265.0,265.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,397.5,265.0,0.0,-2,0,0,0,0,0,0,2,530,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
2,143869,172.16.0.5-192.168.50.4-648-40660-17,172.16.0.5,648,192.168.50.4,40660,17,2018-11-03 10:09:00.565608,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
3,16171,172.16.0.5-192.168.50.4-872-54308-17,172.16.0.5,872,192.168.50.4,54308,17,2018-11-03 10:09:00.565993,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
4,80845,172.16.0.5-192.168.50.4-873-40653-17,172.16.0.5,873,192.168.50.4,40653,17,2018-11-03 10:09:00.565994,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS


In [950]:
MSSQLdf.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,98115,172.16.0.5-192.168.50.4-615-28754-17,172.16.0.5,615,192.168.50.4,28754,17,2018-11-03 10:29:52.072724,3,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,9.813333e+08,666666.666667,3.000000,0.000000,3.0,3.0,3.0,3.000000,0.000000,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,666666.666667,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP
1,137,172.16.0.5-192.168.50.4-0-0-0,172.16.0.5,0,192.168.50.4,0,0,2018-11-03 10:29:52.072729,117876168,25274,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,214.835623,4654.905343,87566.968681,6258062.0,0.0,117876168.0,4664.114589,87654.134461,6258062.0,0.0,114812108.0,2.343104e+06,4.442053e+06,15568642.0,1.0,0,0,0,0,0,0,214.411449,0.424174,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,25274,0,50,0,-1,-1,0,0,81408014.0,0.0,81408014.0,81408014.0,6258062.0,0.0,6258062.0,6258062.0,0,1,LDAP
2,98988,172.16.0.5-192.168.50.4-900-42364-17,172.16.0.5,900,192.168.50.4,42364,17,2018-11-03 10:29:52.072825,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1.472000e+09,1000000.000000,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,1000000.000000,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP
3,35177,172.16.0.5-192.168.50.4-616-10537-17,172.16.0.5,616,192.168.50.4,10537,17,2018-11-03 10:29:52.073221,3,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,9.813333e+08,666666.666667,3.000000,0.000000,3.0,3.0,3.0,3.000000,0.000000,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,666666.666667,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP
4,55362,172.16.0.5-192.168.50.4-617-14928-17,172.16.0.5,617,192.168.50.4,14928,17,2018-11-03 10:29:52.073285,44,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,6.690909e+07,45454.545455,44.000000,0.000000,44.0,44.0,44.0,44.000000,0.000000,44.0,44.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,-2,0,45454.545455,0.000000,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP


In [951]:
NetBIOS.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,302291,172.16.0.5-192.168.50.4-648-16174-17,172.16.0.5,648,192.168.50.4,16174,17,2018-11-03 10:01:48.920574,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
1,341625,172.16.0.5-192.168.50.4-861-34200-17,172.16.0.5,861,192.168.50.4,34200,17,2018-11-03 10:01:48.920625,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
2,245313,172.16.0.5-192.168.50.4-862-4750-17,172.16.0.5,862,192.168.50.4,4750,17,2018-11-03 10:01:48.920685,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
3,266106,172.16.0.5-192.168.50.4-863-4443-17,172.16.0.5,863,192.168.50.4,4443,17,2018-11-03 10:01:48.921008,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,4.580000e+08,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2.000000e+06,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
4,47182,172.16.0.5-192.168.50.4-864-48627-17,172.16.0.5,864,192.168.50.4,48627,17,2018-11-03 10:01:48.921010,48,2,0,458.0,0.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,0.0,9.541667e+06,4.166667e+04,48.0,0.0,48.0,48.0,48.0,48.0,0.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,4.166667e+04,0.0,229.0,229.0,229.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,343.5,229.0,0.0,-2,0,0,0,0,0,0,2,458,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS


In [952]:
Portmap.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,24,192.168.50.254-224.0.0.5-0-0-0,192.168.50.254,0,224.0.0.5,0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.393161,2.601295e+06,4.295632e+06,10001143.0,1.0,114456999.0,2.601295e+06,4.295632e+06,10001143.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.393161,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,0.00,0.000000,0.0,0,0,0,0,0,0,0,45,0,0,0,-1,-1,0,0,8185.583333,28337.112288,98168.0,3.0,9529897.25,351582.631269,10001143.0,9048097.0,0,0,BENIGN
1,26,192.168.50.253-224.0.0.5-0-0-0,192.168.50.253,0,224.0.0.5,0,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.489735,2.079046e+06,3.956754e+06,9978130.0,0.0,114347504.0,2.079046e+06,3.956754e+06,9978130.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.489735,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,0.00,0.000000,0.0,0,0,0,0,0,0,0,56,0,0,0,-1,-1,0,0,35028.416667,121314.911865,420255.0,4.0,9493929.75,351541.079539,9978130.0,8820294.0,0,0,BENIGN
2,176563,172.217.10.98-192.168.50.6-443-54799-6,192.168.50.6,54799,172.217.10.98,443,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333333,20.655911,46.0,46.0,46.0,0.0,5.708722,0.219566,5.205068e+06,1.374380e+07,36373056.0,1.0,36435473.0,7.287095e+06,1.625957e+07,36373056.0,1.0,2.0,2.0,0.0,2.0,2.0,1,0,0,0,120,40,0.164675,0.054892,6.0,46.0,28.222222,21.081851,444.444444,0,0,1,0,0,1,0,0,0.0,31.75,19.333333,46.0,120,0,0,0,0,0,0,6,116,2,92,257,250,5,20,62416.000000,0.000000,62416.0,62416.0,36373056.00,0.000000,36373056.0,36373056.0,0,0,BENIGN
3,50762,172.217.7.2-192.168.50.6-443-54800-6,192.168.50.6,54800,172.217.7.2,443,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333333,20.655911,46.0,46.0,46.0,0.0,5.708843,0.219571,5.204958e+06,1.374351e+07,36372291.0,1.0,36434705.0,7.286941e+06,1.625923e+07,36372291.0,1.0,2.0,2.0,0.0,2.0,2.0,1,0,0,0,120,40,0.164678,0.054893,6.0,46.0,28.222222,21.081851,444.444444,0,0,1,0,0,1,0,0,0.0,31.75,19.333333,46.0,120,0,0,0,0,0,0,6,116,2,92,253,252,5,20,62413.000000,0.000000,62413.0,62413.0,36372291.00,0.000000,36372291.0,36372291.0,0,0,BENIGN
4,87149,172.217.10.98-192.168.50.6-443-54801-6,192.168.50.6,54801,172.217.10.98,443,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333333,20.655911,46.0,46.0,46.0,0.0,5.708855,0.219571,5.204947e+06,1.374348e+07,36372216.0,1.0,36434626.0,7.286925e+06,1.625919e+07,36372216.0,1.0,2.0,2.0,0.0,2.0,2.0,1,0,0,0,120,40,0.164679,0.054893,6.0,46.0,28.222222,21.081851,444.444444,0,0,1,0,0,1,0,0,0.0,31.75,19.333333,46.0,

In [953]:
Syn_2.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,445444,172.16.0.5-192.168.50.4-9429-9429-6,172.16.0.5,9429,192.168.50.4,9429,6,2018-11-03 11:36:28.607338,36063894,7,2,42.0,12.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.497342e+00,2.495571e-01,4.507987e+06,7.062714e+06,18628035.0,1.0,36063894.0,6.010649e+06,7.680923e+06,18628035.0,1.0,2.0,2.000000e+00,0.000000e+00,2.0,2.0,0,0,0,0,140,40,1.940999e-01,0.055457,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,6.666667,6.0,6.0,140,0,0,0,0,0,0,7,42,2,12,5840,0,6,20,18.0,29.444864,52.0,1.0,1.202128e+07,6.253623e+06,18628035.0,6193840.0,0,1,Syn
1,113842,172.16.0.5-192.168.50.4-60224-60224-6,172.16.0.5,60224,192.168.50.4,60224,6,2018-11-03 11:36:28.607339,44851366,8,4,48.0,24.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.605302e+00,2.675504e-01,4.077397e+06,9.056967e+06,28934293.0,1.0,44851314.0,6.407331e+06,1.092235e+07,28934293.0,1.0,3525902.0,1.175301e+06,2.035679e+06,3525900.0,1.0,0,0,0,0,160,80,1.783669e-01,0.089183,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,6.500000,6.0,6.0,160,0,0,0,0,0,0,8,48,4,24,5840,0,7,20,1.0,0.000000,1.0,1.0,2.066268e+07,1.169783e+07,28934293.0,12391060.0,0,1,Syn
2,176377,172.16.0.5-192.168.50.4-33827-11746-6,192.168.50.4,11746,172.16.0.5,33827,6,2018-11-03 11:36:28.607388,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,1.200000e+07,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.000000,6.0,6.0,6.0,0.0,0.0,0,1,0,0,0,1,0,0,0.0,9.000000,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,0,-1,1,20,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,Syn
3,24777,172.16.0.5-192.168.50.4-33828-1431-6,172.16.0.5,33828,192.168.50.4,1431,6,2018-11-03 11:36:28.607391,0,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,inf,inf,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,40,0,0.000000e+00,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,5840,-1,1,20,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
4,85100,172.16.0.5-192.168.50.4-5311-5311-6,172.16.0.5,5311,192.168.50.4,5311,6,2018-11-03 11:36:28.607442,35731470,8,2,48.0,12.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.679192e+00,2.798653e-01,3.970163e+06,6.026604e+06,13693985.0,0.0,35731470.0,5.104496e+06,6.455349e+06,13693985.0,0.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,0,0,0,0,160,40,2.238923e-01,0.055973,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,6.600000,6.0,6.0,160,0,0,0,0,0,0,8,48,2,12,5840,0,7,20,24.5,33.234019,48.0,1.0,1.191047e+07,1.849493e+06,13693985.0,10001398.0,0,1,Syn


In [954]:
UDPdf.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,15798,172.16.0.5-192.168.50.4-9401-15931-17,172.16.0.5,9401,192.168.50.4,15931,17,2018-11-03 10:42:57.176671,1,2,0,2560.0,0.0,1280.0,1280.0,1280.0,0.0,0.0,0.0,0.0,0.0,2.560000e+09,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,1280.0,1280.0,1280.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,1920.0,1280.0,0.0,-2,0,0,0,0,0,0,2,2560,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
1,110891,172.16.0.5-192.168.50.4-9402-29997-17,172.16.0.5,9402,192.168.50.4,29997,17,2018-11-03 10:42:57.176673,0,2,0,816.0,0.0,408.0,408.0,408.0,0.0,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,0.0,0.0,408.0,408.0,408.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,612.0,408.0,0.0,-2,0,0,0,0,0,0,2,816,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
2,66956,172.16.0.5-192.168.50.4-9403-29887-17,172.16.0.5,9403,192.168.50.4,29887,17,2018-11-03 10:42:57.176727,1,2,0,810.0,0.0,405.0,405.0,405.0,0.0,0.0,0.0,0.0,0.0,8.100000e+08,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,405.0,405.0,405.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,607.5,405.0,0.0,-2,0,0,0,0,0,0,2,810,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
3,66144,172.16.0.5-192.168.50.4-9404-7393-17,172.16.0.5,9404,192.168.50.4,7393,17,2018-11-03 10:42:57.176729,1,2,0,852.0,0.0,426.0,426.0,426.0,0.0,0.0,0.0,0.0,0.0,8.520000e+08,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,426.0,426.0,426.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,639.0,426.0,0.0,-2,0,0,0,0,0,0,2,852,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
4,72903,172.16.0.5-192.168.50.4-9405-57957-17,172.16.0.5,9405,192.168.50.4,57957,17,2018-11-03 10:42:57.177121,1,2,0,1240.0,0.0,620.0,620.0,620.0,0.0,0.0,0.0,0.0,0.0,1.240000e+09,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,620.0,620.0,620.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,930.0,620.0,0.0,-2,0,0,0,0,0,0,2,1240,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL


In [955]:
UDPLag_2.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,50880,172.16.0.5-192.168.50.4-35468-49856-17,172.16.0.5,35468,192.168.50.4,49856,17,2018-11-03 11:01:43.652742,47,2,0,778.0,0.0,389.0,389.0,389.000000,0.000000,0.0,0.0,0.0,0.0,1.655319e+07,42553.191489,47.000000,0.000000,47.0,47.0,47.0,47.000000,0.000000,47.0,47.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,42553.191489,0.0,389.0,389.0,389.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,583.5,389.000000,0.0,-2,0,0,0,0,0,0,2,778,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP
1,83164,172.16.0.5-192.168.50.4-44167-44225-17,172.16.0.5,44167,192.168.50.4,44225,17,2018-11-03 11:01:43.653107,2,2,0,786.0,0.0,393.0,393.0,393.000000,0.000000,0.0,0.0,0.0,0.0,3.930000e+08,1000000.000000,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.000000,0.0,393.0,393.0,393.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,589.5,393.000000,0.0,-2,0,0,0,0,0,0,2,786,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP
2,49389,172.16.0.5-192.168.50.4-36215-28771-17,172.16.0.5,36215,192.168.50.4,28771,17,2018-11-03 11:01:43.653383,2,2,0,748.0,0.0,374.0,374.0,374.000000,0.000000,0.0,0.0,0.0,0.0,3.740000e+08,1000000.000000,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.000000,0.0,374.0,374.0,374.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,561.0,374.000000,0.0,-2,0,0,0,0,0,0,2,748,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP
3,34687,172.16.0.5-192.168.50.4-44168-43679-17,172.16.0.5,44168,192.168.50.4,43679,17,2018-11-03 11:01:43.653386,2,2,0,786.0,0.0,393.0,393.0,393.000000,0.000000,0.0,0.0,0.0,0.0,3.930000e+08,1000000.000000,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.000000,0.0,393.0,393.0,393.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,589.5,393.000000,0.0,-2,0,0,0,0,0,0,2,786,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP
4,87466,172.16.0.5-192.168.50.4-52334-44960-17,172.16.0.5,52334,192.168.50.4,44960,17,2018-11-03 11:01:43.653387,880701,18,0,6600.0,0.0,401.0,330.0,366.666667,28.171742,0.0,0.0,0.0,0.0,7.494030e+03,20.438265,51805.941176,56640.323946,111083.0,1.0,880701.0,51805.941176,56640.323946,111083.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,96,0,20.438265,0.0,330.0,401.0,364.736842,28.641156,820.315789,0,0,0,0,0,0,0,0,0.0,385.0,366.666667,0.0,96,0,0,0,0,0,0,18,6600,0,0,-1,-1,17,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP


Como se puede observar, todos los datasets tienen 80 features por lo que podemos asumir que son los mismos features en todos los datasets, por lo que no es necesario hacer un análisis de features por dataset, sino que se puede hacer un análisis de features en general. Para ello primero se juntarán todos los datasets en uno solo.

In [956]:
# comprobar que todos los datasets tengan las mismas columnas
columns = DrDoS_DNS.columns
for df in [DrDoS_LDAP, DrDoS_MSSQL, DrDoS_NetBIOS, DrDoS_NTP, DrDoS_SNMP, DrDoS_SSDP, DrDoS_UDP, Syn, TFTPdf, UDPLag]:
    if not all(df.columns == columns):
        print('Columns do not match!')
        break

#### Unión de los datasets

Se unen los datasets de entrenamiento y testeo en un solo dataset para poder hacer el pre-procesamiento y la exploración de datos.

In [957]:
# Concatenamos los DataFrames en uno solo de 3 en 3
Concatfull = pd.concat([DrDoS_DNS, DrDoS_LDAP, DrDoS_MSSQL])

In [958]:
# Concatenamos los DataFrames en uno solo de 3 en 3
Concatfull = pd.concat([DrDoS_NetBIOS, DrDoS_NTP, DrDoS_SNMP])

In [959]:
# Concatenamos los DataFrames en uno solo de 3 en 3
Concatfull = pd.concat([DrDoS_SSDP, DrDoS_UDP, Syn])

In [960]:
# Concatenamos los DataFrames en uno solo de 3 en 3
Concatfull = pd.concat([TFTPdf, UDPLag, LDAPdf])

In [961]:
# Concatenamos los DataFrames en uno solo de 3 en 3
Concatfull = pd.concat([MSSQLdf, NetBIOS, Portmap])

In [962]:
# Concatenamos los DataFrames en uno solo de 3 en 3
Concatfull = pd.concat([Syn_2, UDPdf, UDPLag_2])


In [963]:
# Guardamos el DataFrame completo en un archivo csv
Concatfull.to_csv('archivo_completo.csv', index=False)

### Luego de que todo este en un mismo dataset, procede a hacer el preprocesamiento

In [964]:
filename = 'archivo_completo.csv'

data = pd.read_csv(filename, index_col=False)
# data = pd.read_csv(filename, index_col=False, nrows=10000)
print('Dataframe shape',data.shape)
data.head()

C:\Users\julio\AppData\Local\Temp\ipykernel_20036\2399804169.py:3: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename, index_col=False)


Dataframe shape (300000, 88)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,445444,172.16.0.5-192.168.50.4-9429-9429-6,172.16.0.5,9429,192.168.50.4,9429,6,2018-11-03 11:36:28.607338,36063894,7,2,42.0,12.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.497342e+00,2.495571e-01,4.507987e+06,7.062714e+06,18628035.0,1.0,36063894.0,6.010649e+06,7.680923e+06,18628035.0,1.0,2.0,2.000000e+00,0.000000e+00,2.0,2.0,0,0,0,0,140,40,1.940999e-01,0.055457,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,6.666667,6.0,6.0,140,0,0,0,0,0,0,7,42,2,12,5840,0,6,20,18.0,29.444864,52.0,1.0,1.202128e+07,6.253623e+06,18628035.0,6193840.0,0,1,Syn
1,113842,172.16.0.5-192.168.50.4-60224-60224-6,172.16.0.5,60224,192.168.50.4,60224,6,2018-11-03 11:36:28.607339,44851366,8,4,48.0,24.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.605302e+00,2.675504e-01,4.077397e+06,9.056967e+06,28934293.0,1.0,44851314.0,6.407331e+06,1.092235e+07,28934293.0,1.0,3525902.0,1.175301e+06,2.035679e+06,3525900.0,1.0,0,0,0,0,160,80,1.783669e-01,0.089183,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,6.500000,6.0,6.0,160,0,0,0,0,0,0,8,48,4,24,5840,0,7,20,1.0,0.000000,1.0,1.0,2.066268e+07,1.169783e+07,28934293.0,12391060.0,0,1,Syn
2,176377,172.16.0.5-192.168.50.4-33827-11746-6,192.168.50.4,11746,172.16.0.5,33827,6,2018-11-03 11:36:28.607388,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,1.200000e+07,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.000000,6.0,6.0,6.0,0.0,0.0,0,1,0,0,0,1,0,0,0.0,9.000000,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,0,-1,1,20,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,Syn
3,24777,172.16.0.5-192.168.50.4-33828-1431-6,172.16.0.5,33828,192.168.50.4,1431,6,2018-11-03 11:36:28.607391,0,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,inf,inf,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,40,0,0.000000e+00,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,5840,-1,1,20,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
4,85100,172.16.0.5-192.168.50.4-5311-5311-6,172.16.0.5,5311,192.168.50.4,5311,6,2018-11-03 11:36:28.607442,35731470,8,2,48.0,12.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.679192e+00,2.798653e-01,3.970163e+06,6.026604e+06,13693985.0,0.0,35731470.0,5.104496e+06,6.455349e+06,13693985.0,0.0,1.0,1.000000e+00,0.000000e+00,1.0,1.0,0,0,0,0,160,40,2.238923e-01,0.055973,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,6.600000,6.0,6.0,160,0,0,0,0,0,0,8,48,2,12,5840,0,7,20,24.5,33.234019,48.0,1.0,1.191047e+07,1.849493e+06,13693985.0,10001398.0,0,1,Syn


Se procede a hacer el pre-procesamiento de los datos, para ello se hace un análisis de los datos, se limpian los datos y se seleccionan las features relevantes.

In [965]:
data.describe()

,Unnamed: 0,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound
count,300000.000000,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,2.999970e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,300000.000000,300000.000000,300000.0,300000.0,300000.0,3.000000e+05,300000.000000,3.000000e+05,3.000000e+05,300000.000000,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.0,300000.0000,300000.000000,300000.0,300000.000000,300000.000000,300000.000000,300000.0,300000.000000,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.0,300000.0,300000.0,300000.0,300000.0,300000.0,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,300000.000000
mean,119301.739710,42087.547893,32909.061080,13.283580,4.412941e+06,3.864757,0.609387,934.729373,4.992484e+01,272.464660,254.755447,262.549963,9.079588,2.873870,1.209977,1.632769,0.503993,inf,inf,4.997349e+05,8.559921e+05,2.131941e+06,5.240840e+01,4.410692e+06,6.708193e+05,9.567593e+05,2.130676e+06,4.821608e+01,1.344988e+06,3.028627e+05,5.163199e+05,1.038552e+06,0.588663,0.001083,0.0,0.0,0.0,-3.508026e+07,12.622387,6.850944e+05,2.147333e+03,254.751233,273.556303,261.491196,8.968186,5.301231e+02,0.0,0.0001,0.001083,0.0,0.334570,0.003123,0.001540,0.0,0.091413,361.490525,262.549963,1.632769,-3.508026e+07,0.0,0.0,0.0,0.0,0.0,0.0,3.864757,934.729373,0.609387,4.992484e+01,1980.629030,12.291690,2.837233,-1.514372e+07,1.568321e+04,2.135346e+04,3.801536e+04,2.254143e+03,1.672155e+06,4.735045e+05,2.094670e+06,1.273279e+06,0.993150
std,121451.430303,15077.610110,18875.191614,5.203794,1.235110e+07,4.490261,4.149144,1180.031580,6.320208e+03,203.771749,194.576043,197.640721,16.986088,61.510157,4.572876,18.777913,18.072988,NaN,NaN,1.470336e+06,2.471825e+06,6.077468e+06,9.935350e+03,1.234960e+07,1.890148e+06,2.748646e+06,6.076703e+06,9.970568e+03,6.598753e+06,1.467383e+06,2.528528e+06,5.006877e+06,4.504057,0.032896,0.0,0.0,0.0,3.374486e+08,121.890906,8.769878e+05,1.561694e+04,194.581159,209.858170,197.088635,21.206043,1.209214e+04,0.0,0.0100,0.032896,0.0,0.471841,0.055800,0.039213,0.0,0.298380,285.928550,197.640721,18.777913,3.374486e+08,0.0,0.0,0.0,0.0,0.0,0.0,4.490261,1180.031580,4.149144,6.320208e+03,2888.996604,676.094804,3.471858,1.259534e+08,1.648430e+05,2.112403e+05,3.668797e+0

Vemos mediante la descripción de los datos numéricos que hay columnas que no aportan ningún valor a los datos como "Bwd Packet Length Std", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "FIN Flag Count", "RST Flag Count", "PSH Flag Count", "CWE Flag Count", "ECE Flag Count", "Fwd Avg Bytes/Bulk", "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate" y "SimillarHTTP" en el que todos los valores son 0 o columnas como "Flow ID" y "Unnamed: 0" que son identificadores únicos para cada fila y no aportan nada a los datos.

Luego otras columnas como "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Bwd Header Length", "Min Packet Length", "Max Packet Length", "Packet Length Mean", "Packet Length Std", "Packet Length Variance", "SYN Flag Count", "ACK Flag Count", "URG Flag Count", "Down/Up Ratio", "Subflow Bwd Packets", "Subflow Bwd Bytes" y "Active Std" que tienen valores muy similares en todas las filas, por lo que no aportan mucho a los datos.

In [966]:
# elimina espacios al principio de los nombres de las columnas
data.columns = data.columns.str.strip()

In [967]:
# Eliminar las columnas no utiles.
df = data.drop(columns=['Fwd IAT Min','Bwd IAT Total','Bwd IAT Mean','Bwd IAT Std','Bwd IAT Max','Bwd IAT Min','Bwd Header Length','Min Packet Length','Max Packet Length','Packet Length Mean','Packet Length Std','Packet Length Variance','SYN Flag Count','ACK Flag Count','URG Flag Count','Down/Up Ratio','Subflow Bwd Packets','Subflow Bwd Bytes','Active Std'])
df = data.drop(columns=['Flow ID', 'Unnamed: 0', 'Bwd Packet Length Std', 'SimillarHTTP','Fwd PSH Flags','Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','FIN Flag Count','RST Flag Count','PSH Flag Count','CWE Flag Count','ECE Flag Count','Fwd Avg Bytes/Bulk','Fwd Avg Packets/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate'])

In [968]:
#Elimina todos los inf y NAN para no tener problemas en el futuro
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [969]:
# Verificar si hay valores faltantes en el DataFrame
df.isnull().sum()

Source IP                      0
Source Port                    0
Destination IP                 0
Destination Port               0
Protocol                       0
Timestamp                      0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
Total Length of Bwd Packets    0
Fwd Packet Length Max          0
Fwd Packet Length Min          0
Fwd Packet Length Mean         0
Fwd Packet Length Std          0
Bwd Packet Length Max          0
Bwd Packet Length Min          0
Bwd Packet Length Mean         0
Flow Bytes/s                   0
Flow Packets/s                 0
Flow IAT Mean                  0
Flow IAT Std                   0
Flow IAT Max                   0
Flow IAT Min                   0
Fwd IAT Total                  0
Fwd IAT Mean                   0
Fwd IAT Std                    0
Fwd IAT Max                    0
Fwd IAT Min                    0
Bwd IAT Total                  0
Bwd IAT Me

Se puede ver que ya no hay columnas con valores nulos, pero sí algunas que tienen valores 0, para estas es necesario escalar los datos para algunas columnas, sin embargo otras que el 0 es significativo como los puertos, se dejarán así.

In [970]:
# obtener una lista de todas las columnas donde el valor minimo es 0
zero_cols = [col for col in df.columns if df[col].min() == 0]
print(zero_cols)

['Source Port', 'Destination Port', 'Protocol', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Flow Bytes/s', 'Flow IAT Std', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Bwd Header Length', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'SYN Flag Count', 'ACK Flag Count', 'URG Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'act_data_pkt_fwd', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', '

In [971]:
# eliminar las columnas donde el valor minimo 0 es significativo
zero_cols.remove('Source Port')
zero_cols.remove('Destination Port')
zero_cols.remove('Protocol')

In [972]:
for column in zero_cols:
    print(f"Columna: {column}")
    print(f"Valor máximo: {df[column].max()}")
    print(f"Valor mínimo: {df[column].min()}")

Columna: Total Backward Packets
Valor máximo: 1347
Valor mínimo: 0
Columna: Total Length of Fwd Packets
Valor máximo: 102472.0
Valor mínimo: 0.0
Columna: Total Length of Bwd Packets
Valor máximo: 2466308.0
Valor mínimo: 0.0
Columna: Fwd Packet Length Max
Valor máximo: 3495.0
Valor mínimo: 0.0
Columna: Fwd Packet Length Min
Valor máximo: 1472.0
Valor mínimo: 0.0
Columna: Fwd Packet Length Mean
Valor máximo: 1472.0
Valor mínimo: 0.0
Columna: Fwd Packet Length Std
Valor máximo: 1107.2994856881053
Valor mínimo: 0.0
Columna: Bwd Packet Length Max
Valor máximo: 3621.0
Valor mínimo: 0.0
Columna: Bwd Packet Length Min
Valor máximo: 323.0
Valor mínimo: 0.0
Columna: Bwd Packet Length Mean
Valor máximo: 1830.9636228656263
Valor mínimo: 0.0
Columna: Flow Bytes/s
Valor máximo: 2944000000.0
Valor mínimo: 0.0
Columna: Flow IAT Std
Valor máximo: 66814080.73844741
Valor mínimo: 0.0
Columna: Flow IAT Min
Valor máximo: 2923883.0
Valor mínimo: 0.0
Columna: Fwd IAT Total
Valor máximo: 119995583.0
Valor mín

In [973]:
# Escalar y normalizar las columnas relevantes
scaler = MinMaxScaler()
df[zero_cols] = scaler.fit_transform(df[zero_cols])
scaler = StandardScaler()
df[zero_cols] = scaler.fit_transform(df[zero_cols])
scaler = Normalizer()
df[zero_cols] = scaler.fit_transform(df[zero_cols])

In [974]:
# Convertir la columna 'Timestamp' a un formato de fecha y hora
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')


Por último, la columna label es la que contiene la informacion de que tipo de ataque es o si es benigno, por lo que es necesario convertir los valores de esta columna a valores numéricos para poder usarlos en el modelo. Tambien se debe contar la cantidad de ataques y la cantidad de datos benignos para poder balancear los datos.

In [975]:
# contar la cantidad de cada tipo de ataque
print(df['Label'].value_counts())
min_type = df['Label'].value_counts().min()
print(f"La categoría con menos datos tiene {min_type} muestras")

# convertir la columna 'Label' a valores numericos
df['Label'] = df['Label'].astype('category')
df['Label'] = df['Label'].cat.codes

# contar la cantidad de cada tipo de ataque
print(df['Label'].value_counts())

# balancear el dataset para que tenga la misma cantidad de cada tipo de ataque
df = df.groupby('Label').apply(lambda x: x.sample(n=min_type, random_state=42)).reset_index(drop=True)

# contar la cantidad de cada tipo de ataque
print(df['Label'].value_counts())

UDP       172874
Syn        94002
MSSQL      23597
BENIGN      1740
Name: Label, dtype: int64
La categoría con menos datos tiene 1740 muestras
3    172874
2     94002
1     23597
0      1740
Name: Label, dtype: int64
0    1740
1    1740
2    1740
3    1740
Name: Label, dtype: int64


In [976]:
df.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,SYN Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,192.168.50.6,58410,4.2.2.4,53,17,2018-11-03 10:47:51.786396,20808,2,0.018998,-0.043233,0.000460,-0.069454,-0.067265,-0.068605,-0.031581,0.044791,0.624479,0.150623,-0.035724,1.922338e+02,6936.000000,-0.020139,20803.0,-0.000299,-0.021060,-0.020927,-0.020523,-0.020674,-0.000267,-0.012005,-0.012156,-0.012027,-0.012217,0.017771,64,0.012722,9.611688e+01,-0.007743,-0.067262,-0.063227,-0.066575,-0.001199,-0.002216,-0.000558,-0.040133,-0.003245,0.174350,-0.064436,-0.068605,0.150623,64,2,-0.043233,0.018998,0.000460,-1,-1,-0.031268,32,-0.005601,-0.005951,-0.006100,-0.001476,-0.020045,-0.016444,-0.020289,-0.018805,-0.694433,0
1,192.168.50.6,64028,4.2.2.4,53,17,2018-11-03 10:53:16.572797,20715,2,0.017867,-0.040015,0.000552,-0.063429,-0.061281,-0.062572,-0.029701,0.048261,0.669915,0.161758,-0.033596,1.930968e+02,6905.000000,-0.018941,20709.0,-0.000276,-0.019806,-0.019681,-0.019301,-0.019443,-0.000251,-0.011290,-0.011433,-0.011311,-0.011490,0.028688,64,0.022583,9.654839e+01,-0.007281,-0.061278,-0.057629,-0.060657,-0.001128,-0.002084,-0.000525,-0.037743,-0.003052,0.163969,-0.058917,-0.062572,0.161758,64,2,-0.040015,0.017867,0.000552,-1,-1,-0.029406,32,-0.005268,-0.005597,-0.005737,-0.001388,-0.018852,-0.015465,-0.019081,-0.017685,-0.653082,0
2,192.168.50.9,48704,112.121.107.29,3001,6,2018-11-03 10:49:53.534181,1,2,-0.006788,-0.035430,-0.000365,-0.055414,-0.061052,-0.058377,0.033511,-0.002159,-0.012236,-0.004018,-0.024125,2.000000e+06,1.000000,-0.016026,1.0,-0.000239,-0.016529,-0.016425,-0.016108,-0.016227,-0.000219,-0.009422,-0.009541,-0.009440,-0.009589,-0.006040,64,-0.004785,2.000000e+06,-0.006354,-0.061049,-0.053996,-0.057083,0.018490,-0.000833,-0.000438,-0.031500,0.816290,-0.014172,-0.053903,-0.058377,-0.004018,64,2,-0.035430,-0.006788,-0.000365,341,-1,-0.037553,32,-0.004396,-0.004671,-0.004788,-0.001159,-0.015733,-0.012907,-0.015924,-0.014760,-0.545046,0
3,192.168.50.8,60359,172.217.9.226,443,6,2018-11-03 10:50:16.036324,332,4,-0.006804,-0.031744,-0.000366,-0.052158,-0.059778,-0.054615,0.025471,-0.002164,-0.012265,-0.004027,-0.028057,1.204819e+04,110.666667,-0.016060,306.0,-0.000235,-0.016567,-0.016461,-0.016144,-0.016263,-0.000215,-0.009445,-0.009564,-0.009462,-0.009612,-0.006054,80,-0.004797,1.204819e+04,-0.006370,-0.059775,-0.050838,-0.053872,0.017609,-0.000892,-0.000439,-0.031574,0.818229,-0.014206,-0.051981,-0.054615,-0.004027,80,4,-0.031744,-0.006804,-0.000366,16537,-1,0.001484,20,-0.004407,-0.004682,-0.004799,-0.001162,-0.015770,-0.012937,-0.015962,-0.014795,-0.546341,0
4,192.168.50.9,42220,172.217.2.163,443,6,2018-11-03 10:47:58.663583,117484627,54,0.297608,0.059607,0.031287,0.021382,-0.024140,-0.016202,0.126183,0.409370,-0.004838,0.149558,-0.011087,1.055457e+00,955159.569106,0.047264,58554454.0,-0.000096,0.163033,0.014409,0.066692,0.165525,-0.000088,0.313187,0.016882,0.065982,0.204589,-0.002388,1744,0.327497,4.596346e-01,-0.002512,-0.024139,0

In [977]:
#Se guarda el .csv en limpio listo para usar los modelos
df.to_csv('archivo_completo_limpio.csv', index=False)